In [22]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [23]:
from transformers import  AutoTokenizer, AutoModel, AutoModelForTokenClassification

In [26]:
from onnxruntime.quantization import quantize_dynamic, QuantType
import torch.nn.functional as F
from pathlib import Path
import onnxruntime
import numpy as np
import torch
import onnx
import time
import os
import shutil

In [27]:
model_type = 'tokenizer_onnx' 
model_quant = 'tokenizer_onnx/' + 'model.onnx'

In [28]:
tokenizer = AutoTokenizer.from_pretrained(model_type) 

In [29]:
from transformers import pipeline
import json
from tqdm import tqdm
import pandas as pd
config = json.load(open("config.json"))
id2label = config['id2label']

In [30]:
import numpy as np
import torch
import torch.nn.functional as F

def run_ner_inference(input_text):
    encoded_input = tokenizer(input_text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True)
    input_ids = encoded_input['input_ids'].numpy()
    attention_mask = encoded_input['attention_mask'].numpy()

    ort_inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask
    }

    ort_outs = ort_session.run(['logits'], ort_inputs)
    logits = ort_outs[0]

    probabilities = F.softmax(torch.from_numpy(logits), dim=-1).numpy()

    return probabilities, input_ids

ort_session = onnxruntime.InferenceSession(model_quant, providers=["CPUExecutionProvider"])



In [31]:
def clean_and_combine_tokens(tokens, labels):
    cleaned_tokens = []
    cleaned_labels = []
    current_word = ""
    current_label = None

    for token, label in zip(tokens, labels):
        if token in ["<s>", "</s>", "<unk>"]:  
            continue
        
        if token.startswith("▁"):  
            if current_word:  
                cleaned_tokens.append(current_word)
                cleaned_labels.append(current_label)
            current_word = token[1:]  
            current_label = label
        else:  
            current_word += token

        if current_label == "O":
            current_label = label

    if current_word:
        cleaned_tokens.append(current_word)
        cleaned_labels.append(current_label)

    return cleaned_tokens, cleaned_labels


In [39]:
input_text = "Мен бала күнімде естуші едім, біздің қазақ сартты көрсе, күлуші еді «енеңді ұрайын, кең қолтық, шүлдіреген тәжік, Арқадан үй төбесіне саламын деп, қамыс артқан, бұтадан қорыққан, көз көргенде «әке-үке» десіп, шығып кетсе, қызын боқтасқан, «сарт-сұрт деген осы» деп. Ноғайды көрсе, оны да боқтап күлуші еді: «түйеден қорыққан ноғай, атқа мінсе - шаршап, жаяу жүрсе - демін алады, ноғай дегенше, ноқай десеңші, түкке ыңғайы келмейді, солдат ноғай, қашқын ноғай, башалшік ноғай» деп. Орысқа да күлуші еді: «ауылды көрсе шапқан, жаман сасыр бас орыс» деп.  Орыс ойына келгенін қылады деген... не айтса соған нанады, «ұзын құлақты тауып бер депті» деп.  Сонда мен ойлаушы едім: ей, құдай-ай, бізден басқа халықтың бәрі антұрған, жаман келеді екен, ең тәуір халық біз екенбіз деп, әлгі айтылмыш сөздерді бір үлкен қызық көріп, қуанып күлуші едім.  Енді қарап тұрсам, сарттың екпеген егіні жоқ, шығармаған жемісі жоқ, саудагерінің жүрмеген жері жоқ, қылмаған шеберлігі жоқ. Өзіменен өзі әуре болып, біріменен бірі ешбір шаһары жауласпайды! Орысқа қарамай тұрғанда қазақтың өлісінің ахиреттігін, тірісінің киімін сол жеткізіп тұрды. Әке балаға қимайтұғын малыңды кірелеп сол айдап кетіп тұрды ғой. Орысқа қараған соң да, орыстың өнерлерін бізден олар көп үйреніп кетті. Үлкен байлар да, үлкен молдалар да, ептілік, қырмызылық, сыпайылық - бәрі соларда. Ноғайға қарасам, солдаттыққа да шыдайды, кедейлікке де шыдайды, қазаға да шыдайды, молда, медресе сақтап, дін күтуге де шыдайды. Еңбек қылып, мал табудың да жөнін солар біледі, салтанат, әсем де соларда. Оның малдыларына, құзғын тамағымыз үшін, біріміз жалшы, біріміз қош алушымыз. Біздің ең байымызды: «сәнің шақшы аяғың білән пышыратырға қойған идән түгіл, шық, сасық казақ», - деп үйінен қуып шығарады. Оның бәрі - бірін-бірі қуып қор болмай, шаруа қуып, өнер тауып, мал тауып, зор болғандық әсері. Орысқа айтар сөз де жоқ, біз құлы, күңі құрлы да жоқпыз. Бағанағы мақтан, бағанағы қуанған, күлген сөздеріміз қайда?" 
probabilities, input_ids = run_ner_inference(input_text)

predicted_classes = np.argmax(probabilities, axis=-1)

id2label = config['id2label']

predicted_labels = [id2label[str(class_id)] for class_id in predicted_classes[0]]

tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
clean_tokens, clean_labels = clean_and_combine_tokens(tokens, predicted_labels)
for token, label in zip(clean_tokens, clean_labels):
    print(f"token: {token} \t label: {label}")

token: Мен 	 label: O
token: бала 	 label: O
token: күнімде 	 label: O
token: естуші 	 label: O
token: едім, 	 label: O
token: біздің 	 label: O
token: қазақ 	 label: B-NORP
token: сартты 	 label: O
token: көрсе, 	 label: O
token: күлуші 	 label: O
token: еді 	 label: O
token: «енеңді 	 label: O
token: ұрайын, 	 label: O
token: кең 	 label: O
token: қолтық, 	 label: O
token: шүлдіреген 	 label: O
token: тәжік, 	 label: O
token: Арқадан 	 label: B-LOCATION
token: үй 	 label: O
token: төбесіне 	 label: O
token: саламын 	 label: O
token: деп, 	 label: O
token: қамыс 	 label: O
token: артқан, 	 label: O
token: бұтадан 	 label: O
token: қорыққан, 	 label: O
token: көз 	 label: O
token: көргенде 	 label: O
token: «әке-үке» 	 label: O
token: десіп, 	 label: O
token: шығып 	 label: O
token: кетсе, 	 label: O
token: қызын 	 label: O
token: боқтасқан, 	 label: O
token: «сарт-сұрт 	 label: O
token: деген 	 label: O
token: осы» 	 label: O
token: деп. 	 label: O
token: Ноғайды 	 label: O
token: көр